In [ ]:
import torch
import random
from trl import AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
from transformers import AutoTokenizer
from utils import load_prompts, evaluation

In [ ]:
MODEL_PATH = 'logs/POEM_GPT2_BERT_output128_ND_IN_0.7/EPOCH400'

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token

output_min_length = 4
output_max_length = 128
output_length_sampler = LengthSampler(output_min_length, output_max_length)

device = 0 if torch.cuda.is_available() else 'cpu'
model.to(device)

In [ ]:
generation_kwargs = {
        "min_length": -1,
        "top_k": 0.0,
        "top_p": 0.9,
        "temperature": 1.0,
        "do_sample": True,
        "repetition_penalty": 1.0,
        "pad_token_id": tokenizer.eos_token_id,
    }

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}
gen_len = output_length_sampler()
generation_kwargs["max_new_tokens"] = gen_len

In [ ]:
prompts = random.choice(load_prompts(5))
encoded_prompts = torch.tensor(tokenizer.encode(prompts)).long().to(device) 
response = model.generate(input_ids=encoded_prompts.unsqueeze(0), **generation_kwargs)
decoded_response = tokenizer.decode(response.squeeze())
eva = evaluation(decoded_response)
print(prompts)
print(decoded_response)
print(eva)

In [ ]:
decoded_response